# Group Exercise - Association Rules
# Team GIA - Anum, Emilie, Mr Brice and Zuzanna

### Objective: Use the Apriori Algorithm to recommend what permissions a new user can request to the Murex system based on the user's function. Murex is a system within the Investment Bank which has trade capture, risk management and confirmation capabilities. We are aware that the Apriori Alogorithm has some drawbacks in terms of computational processing time/memory and hence we recognise that our objective may not be met fully.

## 1) Libraries

In [1]:
## import libraries

import pandas as pd
import numpy as np

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

## 2) Import data

In [16]:
## Import Murex data - this excel file contains a system access report which contains details of user departments/functions
## and the permissions that they have to the Murex system.
file = 'Murex_data.xlsx'

In [17]:
xl=pd.ExcelFile(file)

In [18]:
print(xl.sheet_names)

['Sheet1', 'Sheet2', 'Sheet3']


In [19]:
## Setting murex_raw dataframe as sheet 1
murex_raw = xl.parse('Sheet1')

In [20]:
## sense check Murex data imported - have only included the header row due to data privacy constraints
murex_raw.head(0)

Access Right Owner GPN  Access Right Owner Delegate GPIN Platform Name  \
0                43507541                          43406928  Applications   
1                43507541                          43406928  Applications   
2                43507541                          43406928  Applications   
3                43507541                          43406928  Applications   
4                43507541                          43406928  Applications   

   Asset Asset GUID Owner or Delegate Permission Permission GUID  \
0  Murex  APA148227             Owner      ADMIN      ADRG175643   
1  Murex  APA148227             Owner      ADMIN      ADRG175643   
2  Murex  APA148227             Owner      ADMIN      ADRG175643   
3  Murex  APA148227             Owner      ADMIN      ADRG175643   
4  Murex  APA148227             Owner      ADMIN      ADRG175643   

       Account Owner Name                                         Account Id  \
0          Abhishek Kumar       kumaraau, 43490697, abhishek-i.kumar@ubs.com   
1            Ashish Misra        misraash, 43505615, ashish-za.misra@ubs.com   
2            Ayush Dargar            dargara, 43397293, ayush.dargar@ubs.com   
3  Bharathiraja Rangadoss  rangadob, 43405212, bharathiraja.rangadoss@ubs...   
4           Brajesh Kumar         kumabra, 43395304, brajesh-b.kumar@ubs.com   

   ... Re-hire Date                       Old Function  \
0  ...          NaT                       GPS IBIT FRC   
1  ...          NaT  IB Tech-FRC R&C-RCMT CAL Desk Dev   
2  ...   2014-01-27                       GPS IBIT FRC   
3  ...          NaT                       GPS IBIT FRC   
4  ...          NaT                       GPS IBIT FRC   

                          Old Cost Center  Last Move/Leave Date Job Code  \
0      BSC - ASG IB FRC RC RISK PUNE C288   2017-04-02 01:59:30   EXT021   
1  IB TECH-FRC R&C-RCMT CAL DESK DEV-4001   2017-05-25 01:15:45   EXT021   
2                      ASG IB FRC FX 1584   2017-04-02 01:43:20   EXT021   
3                   GPS UK IB IT FRC 4001   2017-04-02 01:23:39   EXT021   
4                   GPS UK IB IT FRC 4001   2017-04-02 01:40:25   EXT021   

                   Permission Description  \
0  IT - Middleware Administration Monitor   
1  IT - Middleware Administration Monitor   
2  IT - Middleware Administration Monitor   
3  IT - Middleware Administration Monitor   
4  IT - Middleware Administration Monitor   

                         Permission Long Description AccessType  \
0  Murex middleware administrator; This group is ...      Admin   
1  Murex middleware administrator; This group is ...      Admin   
2  Murex middleware administrator; This group is ...      Admin   
3  Murex middleware administrator; This group is ...      Admin   
4  Murex middleware administrator; This group is ...      Admin   

                                        Common Usage Cross Border Restriction  
0  Murex Administration, GPS Team. NoAccess to Mu...                        N  
1  Murex Administration, GPS Team. NoAccess to Mu...                        N  
2  Murex Administration, GPS Team. NoAccess to Mu...                        N  
3  Murex Administration, GPS Team. NoAccess to Mu...                        N  
4  Murex Administration, GPS Team. NoAccess to Mu...                        N  

[5 rows x 64 columns]

## 3) Explore and Transform

In [21]:
## Merging the permissions fields to one concatentated field. 
## Further the Function (i.e. an indicator of the user's department) is prefixed with 'Function -' so that it is easier to
##filter by function later.
murex_raw['Permission_Dept'] = murex_raw['AccessType'] + ' - ' + murex_raw['Permission Description']
murex_raw['Function_v2'] = 'Function - '+murex_raw['Function']


       Access Right Owner GPN  Access Right Owner Delegate GPIN Platform Name  \
0                    43507541                          43406928  Applications   
1                    43507541                          43406928  Applications   
2                    43507541                          43406928  Applications   
3                    43507541                          43406928  Applications   
4                    43507541                          43406928  Applications   
5                    43507541                          43406928  Applications   
6                    43507541                          43406928  Applications   
7                    43507541                          43406928  Applications   
8                    43507541                          43406928  Applications   
9                    43507541                          43406928  Applications   
10                   43507541                          43406928  Applications   
11                   4350754

In [22]:
## Sense checking the data - please note that we have only included the header row due to data privacy constraints
murex_raw.head(0)

Access Right Owner GPN  Access Right Owner Delegate GPIN Platform Name  \
0                43507541                          43406928  Applications   
1                43507541                          43406928  Applications   
2                43507541                          43406928  Applications   
3                43507541                          43406928  Applications   
4                43507541                          43406928  Applications   

   Asset Asset GUID Owner or Delegate Permission Permission GUID  \
0  Murex  APA148227             Owner      ADMIN      ADRG175643   
1  Murex  APA148227             Owner      ADMIN      ADRG175643   
2  Murex  APA148227             Owner      ADMIN      ADRG175643   
3  Murex  APA148227             Owner      ADMIN      ADRG175643   
4  Murex  APA148227             Owner      ADMIN      ADRG175643   

       Account Owner Name                                         Account Id  \
0          Abhishek Kumar       kumaraau, 43490697, abhishek-i.kumar@ubs.com   
1            Ashish Misra        misraash, 43505615, ashish-za.misra@ubs.com   
2            Ayush Dargar            dargara, 43397293, ayush.dargar@ubs.com   
3  Bharathiraja Rangadoss  rangadob, 43405212, bharathiraja.rangadoss@ubs...   
4           Brajesh Kumar         kumabra, 43395304, brajesh-b.kumar@ubs.com   

   ...                         Old Cost Center Last Move/Leave Date  Job Code  \
0  ...      BSC - ASG IB FRC RC RISK PUNE C288  2017-04-02 01:59:30    EXT021   
1  ...  IB TECH-FRC R&C-RCMT CAL DESK DEV-4001  2017-05-25 01:15:45    EXT021   
2  ...                      ASG IB FRC FX 1584  2017-04-02 01:43:20    EXT021   
3  ...                   GPS UK IB IT FRC 4001  2017-04-02 01:23:39    EXT021   
4  ...                   GPS UK IB IT FRC 4001  2017-04-02 01:40:25    EXT021   

                   Permission Description  \
0  IT - Middleware Administration Monitor   
1  IT - Middleware Administration Monitor   
2  IT - Middleware Administration Monitor   
3  IT - Middleware Administration Monitor   
4  IT - Middleware Administration Monitor   

                         Permission Long Description  AccessType  \
0  Murex middleware administrator; This group is ...       Admin   
1  Murex middleware administrator; This group is ...       Admin   
2  Murex middleware administrator; This group is ...       Admin   
3  Murex middleware administrator; This group is ...       Admin   
4  Murex middleware administrator; This group is ...       Admin   

                                        Common Usage Cross Border Restriction  \
0  Murex Administration, GPS Team. NoAccess to Mu...                        N   
1  Murex Administration, GPS Team. NoAccess to Mu...                        N   
2  Murex Administration, GPS Team. NoAccess to Mu...                        N   
3  Murex Administration, GPS Team. NoAccess to Mu...                        N   
4  Murex Administration, GPS Team. NoAccess to Mu...                        N   

                                  Permission_Dept  \
0  Admin - IT - Middleware Administration Monitor   
1  Admin - IT - Middleware Administration Monitor   
2  Admin - IT - Middleware Administration Monitor   
3  Admin - IT - Middleware Administration Monitor   
4  Admin - IT - Middleware Administration Monitor   

                      Function_v2  
0  Function - RES_IB FRC R&C Risk  
1  Function - RES_IB FRC R&C Risk  
2  Function - RES_IB FRC R&C Risk  
3  Function - RES_IB FRC R&C Risk  
4  Function - RES_IB FRC R&C Risk  

[5 rows x 66 columns]

In [30]:
## one data set created for GPN (unique employee code) and the newly created permissions field 'Permission_Dept'
murex_limited1 = murex_raw[['GPN','Permission_Dept']]

In [31]:
## second data set created for GPN (unique employee code) and the newly created concatenated function field 'Function_v2'
murex_limited2 = murex_raw[['GPN','Function_v2']]

## combine data set 1 and 2 to have a consolidated data set showing users, functions and permissions
murex_limited2 = murex_limited2.rename(index=str, columns={"Function_v2": "Permission_Dept"})

In [32]:
murex_limited3 = murex_limited1.append(murex_limited2)

In [33]:
## sense check the data - please note that we have only included the header row due to data privacy constraints
murex_limited3.head(0)

GPN                                 Permission_Dept
0  43490697  Admin - IT - Middleware Administration Monitor
1  43505615  Admin - IT - Middleware Administration Monitor
2  43397293  Admin - IT - Middleware Administration Monitor
3  43405212  Admin - IT - Middleware Administration Monitor
4  43395304  Admin - IT - Middleware Administration Monitor

In [34]:
## Remove dups

## first sort data
murex_limited3.sort_values("GPN",inplace=True)
murex_limited3.head()

GPN                         Permission_Dept
9520  15914                       Function - FCS CC
9520  15914  Read - Middle Office - FRC (Read Only)
7385  15914                       Function - FCS CC
3115  15914  Read - Middle Office - FRC (Read Only)
5250  15914                       Function - FCS CC

In [35]:
# dropping duplicate values 
murex_limited_dedup = murex_limited3.drop_duplicates(['GPN','Permission_Dept']) 

In [36]:
## sense check the data - no need to anonimise as no employee names are visible
murex_limited_dedup.head(1)

GPN                          Permission_Dept
9520   15914                        Function - FCS CC
9520   15914   Read - Middle Office - FRC (Read Only)
10547  27834         Function - Macro High Touch EMEA
10547  27834                      Write - Sales - FRC
9507   31244  Function - GALM Optimisation Costs (fn)

In [37]:
## Now transform the data into an array so that transaction encoder can be applied 
murex_array = murex_limited_dedup.groupby(['GPN']).Permission_Dept.apply(np.array).reset_index()
## sense check the data - no need to anonimise as no employee names are visible
murex_array.head(1)

GPN                                    Permission_Dept
0  15914  [Function - FCS CC, Read - Middle Office - FRC...
1  27834  [Function - Macro High Touch EMEA, Write - Sal...
2  31244  [Function - GALM Optimisation Costs (fn), Read...
3  33710  [Write - Sales - FRC, Function - Credit High T...
4  50859  [Function - GALM COO COSTS (FN), Read - Offsho...

In [38]:
## transaction encoder is named as 'te'
te = TransactionEncoder()

In [39]:
## transaction encoder is applied to fit array data so that we have product name in array
te_ary = te.fit(murex_array['Permission_Dept']).transform(murex_array['Permission_Dept'])

In [40]:
## convert to a dataframe
df = pd.DataFrame(te_ary, columns = te.columns_)

In [41]:
## sense check the data - no need to anonimise as no employee names are visible
df.head(1)

Admin - IT - Administration Houskeeping  \
0                                    False   
1                                    False   
2                                    False   
3                                    False   
4                                    False   

   Admin - IT - Administration Installation  \
0                                     False   
1                                     False   
2                                     False   
3                                     False   
4                                     False   

   Admin - IT - Administration License  Admin - IT - Close-Down APAC  \
0                                False                         False   
1                                False                         False   
2                                False                         False   
3                                False                         False   
4                                False                         False   

   Admin - IT - Close-Down EMEA  Admin - IT - Close-Down US  \
0                         False                       False   
1                         False                       False   
2                         False                       False   
3                         False                       False   
4                         False                       False   

   Admin - IT - Interest Fixings Analysis  Admin - IT - MW Inbound Analysis  \
0                                   False                             False   
1                                   False                             False   
2                                   False                             False   
3                                   False                             False   
4                                   False                             False   

   Admin - IT - Middleware Administration Monitor  \
0                                           False   
1                                           False   
2                                           False   
3                                           False   
4                                           False   

   Admin - IT - Murex Query Service Authorization  ...  \
0                                           False  ...   
1                                           False  ...   
2                                           False  ...   
3                                           False  ...   
4                                           False  ...   

   Write - Trading Swaps - JPY - Trader  Write - Trading Swaps - NZD - Trader  \
0                                 False                                 False   
1                                 False                                 False   
2                                 False                                 False   
3                                 False                                 False   
4                                 False                                 False   

   Write - Trading Swaps - USD - Trader  \
0                                 False   
1                                 False   
2                                 False   
3                                 False   
4                                 False   

   Write - Trading Swaps_Bonds - CHF - Trader  \
0                                       False   
1                                       False   
2                                       False   
3                                       False   
4                                       False   

   Write - Trading Swaps_Bonds - EUR - Trader  \
0                                       False   
1                                       False   
2                                       False   
3                                       False   
4                                       False   

   Write - Trading Swaps_Bonds - GBP - Trader  \
0                                       False   
1                                       False   
2    

In [42]:
## convert to boolean so we can use the apriori algorithm
te_ary = te_ary.astype("int")

In [43]:
## check successful transformation
te_ary

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [44]:
## again apply pandas dataframe and rename columns
te_ary_bol = pd.DataFrame(te_ary, columns = te.columns_)

In [45]:
## sense check the data - no need to anonimise as no employee names are visible
te_ary_bol.head(1)

Admin - IT - Administration Houskeeping  \
0                                           0   
1                                           0   
2                                           0   
3                                           0   
4                                           0   
5                                           0   
6                                           0   
7                                           0   
8                                           0   
9                                           0   
10                                          0   
11                                          0   
12                                          0   
13                                          0   
14                                          0   
15                                          0   
16                                          0   
17                                          0   
18                                          0   
19                                          0   
20                                          0   
21                                          0   
22                                          0   
23                                          0   
24                                          0   
25                                          0   
26                                          0   
27                                          0   
28                                          0   
29                                          0   
...                                       ...   
1308                                        0   
1309                                        0   
1310                                        0   
1311                                        0   
1312                                        0   
1313                                        0   
1314                                        0   
1315                                        0   
1316                                        0   
1317                                        0   
1318                                        0   
1319                                        0   
1320                                        0   
1321                                        0   
1322                                        0   
1323                                        0   
1324                                        0   
1325                                        0   
1326                                        0   
1327                                        0   
1328                                        0   
1329                                        0   
1330                                        0   
1331                                        0   
1332                                        1   
1333                                        0   
1334                                        0   
1335                                        0   
1336                                        0   
1337                                        0   

      Admin - IT - Administration Installation  \
0                                            0   
1                                            0   
2                                            0   
3                                            0   
4                                            0   
5                                            0   
6                                            0   
7                                            0   
8                                            0   
9                                            0   
10                                           0   
11                                           0   
12                                           0   
13                                           0   
14                                           0   
15                                           0   
16                                           0   
17                                           0   
18               

## 4) Model and Understand

In [47]:
## Using Apriori to create a dataframe of frequent itemsets
frequent_itemsets = apriori(te_ary_bol, min_support = 0.001, use_colnames = True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
## sense check the data - no need to anonimise as no employee names are visible
frequent_itemsets.head(1)

support                                           itemsets  length
0       0.004484          (Admin - IT - Administration Houskeeping)       1
1       0.004484         (Admin - IT - Administration Installation)       1
2       0.002242              (Admin - IT - Administration License)       1
3       0.009716                     (Admin - IT - Close-Down APAC)       1
4       0.009716                     (Admin - IT - Close-Down EMEA)       1
5       0.009716                       (Admin - IT - Close-Down US)       1
6       0.004484           (Admin - IT - Interest Fixings Analysis)       1
7       0.004484                 (Admin - IT - MW Inbound Analysis)       1
8       0.016442   (Admin - IT - Middleware Administration Monitor)       1
9       0.001495   (Admin - IT - Murex Query Service Authorization)       1
10      0.009716                 (Admin - IT - PnL Acceptance APAC)       1
11      0.009716                 (Admin - IT - PnL Acceptance EMEA)       1
12      0.009716                   (Admin - IT - PnL Acceptance US)       1
13      0.001495   (Admin - Used to apply Group definition changes)       1
14      0.005979            (Admin - Used to check User Population)       1
15      0.023916                   (Function - Asset Movements OTC)       1
16      0.059791             (Function - Asset Movements_Sec Setts)       1
17      0.001495                        (Function - BSC Quants CHN)       1
18      0.001495                       (Function - Basis XCCY EMEA)       1
19      0.001495                (Function - CMBS Secondary Trading)       1
20      0.004484                       (Function - Change & Design)       1
21      0.001495                 (Function - Client Servicing APAC)       1
22      0.006726                     (Function - Confirmations OTC)       1
23      0.016442                  (Function - Confirmations_Sec TP)       1
24      0.005232                (Function - Credit & Stress - EMEA)       1
25      0.001495          (Function - Credit Data Integritry - PSC)       1
26      0.004484              (Function - Credit Data Quality - SG)       1
27      0.007474                (Function - Credit High Touch APAC)       1
28      0.016442            (Function - Credit High Touch Americas)       1
29      0.022422                (Function - Credit High Touch EMEA)       1
...          ...                                                ...     ...
394230  0.001495  (Admin - IT - Close-Down APAC, Admin - IT - Mi...      17
394231  0.001495  (Admin - IT - Close-Down APAC, Admin - IT - Mi...      17
394232  0.001495  (Admin - IT - Close-Down APAC, Admin - IT - Mi...      17
394233  0.001495  (Admin - IT - Close-Down APAC, Admin - IT - Mi...      17
394234  0.001495  (Admin - IT - Close-Down APAC, Admin - IT - Mi...      17
394235  0.001495  (Admin - IT - Close-Down APAC, Admin - IT - Mi...      17
394236  0.002242  (Admin - IT - Close-Down APAC, Admin - IT - Mi...      17
394237  0.001495  (Admin - IT - Close-Down APAC, Admin - IT - Mi...      17
394238  0.001495  (Admin - IT - Close-Down APAC, Admin - IT - Mi...      17
394239  0.001495  (Admin - IT - Close-Down APAC, Admin - IT - Mi...      17
394240  0.001495  (Admin - IT - Close-Down APAC, Admin - IT - Cl...      17
394241  0.001495  (Admin - IT - Close-Down APAC, Admin - IT - Cl...      17
394242  0.001495  (Admin - IT - Close-Down APAC, Admin - IT - Mi...      17
394243  0.001495  (Admin - IT - Close-Down APAC, Admin - IT - Mi...      17
394244  0.001495  (Admin - IT - Close-Down APAC, Admin - IT - Mi...      17
394245  0.001495  (Admin - IT - Close-Down APAC, Admin - IT - Mi...      17
394246  0.001495  (Admin - IT - Close-Down APAC, Admin - IT - Mi...      17
394247  0.001495  (Admin - IT - Close-Down APAC, Admin - IT - Mi...      17
394248  0.001495  (Admin - IT - Close-Down APAC, Admin - IT - Mi...      17
394249  0.001495  (Admin - IT - Close-Down APAC, Admin - IT - Mi...      17
394250  0.001495  (Admin - IT - Middleware Administration

In [48]:
## add filter so we can get > 1 item set
frequent_itemsets_filtered = frequent_itemsets[frequent_itemsets['length'] > 1 ]

In [49]:
# sort the data by asceding support values
frequent_itemsets_filtered = frequent_itemsets_filtered.sort_values(by=['support'],ascending=1)

In [50]:
## sense check the data - no need to anonimise as no employee names are visible
frequent_itemsets_filtered.head(1)

support                                           itemsets  length
197236  0.001495  (Admin - IT - Close-Down APAC, Admin - IT - Ad...       9
250248  0.001495  (Admin - IT - PnL Acceptance APAC, Read - IT -...      10
250247  0.001495  (Admin - IT - PnL Acceptance APAC, Admin - IT ...      10
250246  0.001495  (Admin - IT - PnL Acceptance APAC, Admin - IT ...      10
250245  0.001495  (Admin - IT - PnL Acceptance APAC, Read - IT -...      10
250244  0.001495  (Admin - IT - Middleware Administration Monito...      10
250242  0.001495  (Admin - IT - Middleware Administration Monito...      10
250241  0.001495  (Admin - IT - Middleware Administration Monito...      10
250249  0.001495  (Admin - IT - PnL Acceptance APAC, Read - IT -...      10
250240  0.001495  (Admin - IT - Middleware Administration Monito...      10
250238  0.001495  (Admin - IT - Middleware Administration Monito...      10
250236  0.001495  (Admin - IT - Middleware Administration Monito...      10
250235  0.001495  (Admin - IT - Middleware Administration Monito...      10
250234  0.001495  (Admin - IT - Middleware Administration Monito...      10
250233  0.001495  (Admin - IT - Middleware Administration Monito...      10
250232  0.001495  (Admin - IT - Middleware Administration Monito...      10
250226  0.001495  (Admin - IT - Middleware Administration Monito...      10
250239  0.001495  (Admin - IT - Middleware Administration Monito...      10
250250  0.001495  (Admin - IT - PnL Acceptance APAC, Admin - IT ...      10
250251  0.001495  (Admin - IT - PnL Acceptance APAC, Read - IT -...      10
250252  0.001495  (Admin - IT - PnL Acceptance APAC, Read - IT -...      10
250269  0.001495  (Read - IT - Support - End  User, Admin - IT -...      10
250268  0.001495  (Admin - IT - PnL Acceptance US, Admin - IT - ...      10
250267  0.001495  (Read - IT - Support - End  User, Admin - IT -...      10
250266  0.001495  (Read - IT - Support - End  User, Admin - IT -...      10
250265  0.001495  (Admin - IT - PnL Acceptance APAC, Read - IT -...      10
250264  0.001495  (Admin - IT - PnL Acceptance APAC, Read - IT -...      10
250263  0.001495  (Admin - IT - PnL Acceptance APAC, Read - IT -...      10
250262  0.001495  (Admin - IT - PnL Acceptance APAC, Admin - IT ...      10
250261  0.001495  (Admin - IT - PnL Acceptance APAC, Read - IT -...      10
...          ...                                                ...     ...
336     0.016442  (Read - IT - Support - End  User, Admin - IT -...       2
1582    0.016442  (Read - IT - Support - End  User, Write - IT -...       3
337     0.016442  (Write - IT - Support Configurator Access, Adm...       2
1773    0.016442  (Read - Offshore_Onshore - Trade Queries (Read...       3
632     0.016442  (Read - Finance - FRC (Read Only), Write - Mid...       2
579     0.017937  (Write - Offshore - Middle Office - Sec Client...       2
1708    0.018685  (Write - Sales - CREDIT, Function - Credit Hig...       3
377     0.018685  (Read - Offshore_Onshore - Trade Queries (Read...       2
601     0.019432  (Read - Offshore_Onshore - Trade Queries (Read...       2
713     0.020179  (Write - Sales - CREDIT, Write - Sales - FRC A...       2
479     0.020179  (Read - Offshore_Onshore - Trade Queries (Read...       2
410     0.020179  (Write - Sales - CREDIT, Function - Credit Hig...       2
602     0.020927  (Write - Middle Office - Controls L1, Function...       2
412     0.020927  (Function - Credit High Touch EMEA, Write - Sa...       2
652     0.020927  (Read - Offshore_Onshore - Trade Queries (Read...       2
1814    0.022422  (Read - IT - Support - End  User, Write - IT -...       3
568     0.022422  (Write - IT - Support Configurator Access, Fun...       2
567     0.022422  (Read - IT - Support - End  User, Function - R...       2
520     0.023916  (Function - Macro High Touch Americas, Write -...       2
544     0.024664  (Read - Offshore_Onshore - Trade Queries (Read...       2
522     0.027653  (Function - Macro High Touch EMEA, Writ

In [52]:
#apply the apriori algorithm with min_support = 0.01 and with antecedent_lenth >=1
frequent_itemsets2 = apriori(te_ary_bol, min_support=0.01, use_colnames=True)
rules = association_rules(frequent_itemsets2, metric="support", min_threshold=0.01)
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules = rules[ (rules['antecedent_len'] >= 1)]

In [61]:
## sense check the data - no need to anonimise as no employee names are visible
rules.head(1)

antecedents  \
0                  (Function - RES_IB FRC R&C Risk)   
1  (Admin - IT - Middleware Administration Monitor)   
2                 (Read - IT - Support - End  User)   
3  (Admin - IT - Middleware Administration Monitor)   
4        (Write - IT - Support Configurator Access)   

                                        consequents  antecedent support  \
0  (Admin - IT - Middleware Administration Monitor)            0.022422   
1                  (Function - RES_IB FRC R&C Risk)            0.016442   
2  (Admin - IT - Middleware Administration Monitor)            0.030643   
3                 (Read - IT - Support - End  User)            0.016442   
4  (Admin - IT - Middleware Administration Monitor)            0.029895   

   consequent support   support  confidence       lift  leverage  conviction  \
0            0.016442  0.015695    0.700000  42.572727  0.015326    3.278525   
1            0.022422  0.015695    0.954545  42.572727  0.015326   21.506726   
2            0.016442  0.016442    0.536585  32.634146  0.015939    2.122414   
3            0.030643  0.016442    1.000000  32.634146  0.015939         inf   
4            0.016442  0.016442    0.550000  33.450000  0.015951    2.185683   

   antecedent_len  
0               1  
1               1  
2               1  
3               1  
4               1

In [62]:
##names data set 'rules' which can later be worked with that can be later sorted and exported
df = pd.DataFrame(data=rules)

In [53]:
## sense check the data - no need to anonimise as no employee names are visible
rules.head(1)

antecedents  \
0                     (Function - RES_IB FRC R&C Risk)   
1     (Admin - IT - Middleware Administration Monitor)   
2                    (Read - IT - Support - End  User)   
3     (Admin - IT - Middleware Administration Monitor)   
4           (Write - IT - Support Configurator Access)   
5     (Admin - IT - Middleware Administration Monitor)   
6    (Read - Offshore_Onshore - Trade Queries (Read...   
7                     (Function - Asset Movements OTC)   
8    (Read - Offshore_Onshore - Trade Queries (Read...   
9               (Function - Asset Movements_Sec Setts)   
10                      (Read - Ops - FRC (Read Only))   
11              (Function - Asset Movements_Sec Setts)   
12                            (Write - Sales - CREDIT)   
13             (Function - Credit High Touch Americas)   
14                               (Write - Sales - FRC)   
15             (Function - Credit High Touch Americas)   
16                            (Write - Sales - CREDIT)   
17                 (Function - Credit High Touch EMEA)   
18                 (Function - Credit High Touch EMEA)   
19                               (Write - Sales - FRC)   
20   (Read - Offshore_Onshore - Trade Queries (Read...   
21                                 (Function - FCS IB)   
22                                 (Function - FCS IB)   
23   (Read - Offshore_Onshore - Trade Query and Agg...   
24   (Read - Offshore_Onshore - Trade Queries (Read...   
25                             (Function - FCS Shared)   
26   (Read - Offshore_Onshore - Trade Query and Agg...   
27                             (Function - FCS Shared)   
28   (Read - Offshore_Onshore - Trade Queries (Read...   
29      (Function - GO RTR Quality Assurance Services)   
..                                                 ...   
206            (Write - IT - TriOptima Account Access)   
207         (Write - IT - Support Configurator Access)   
208  (Read - IT - Support - End  User, Write - IT -...   
209  (Read - IT - Support - End  User, Write - IT -...   
210  (Read - IT - Support - End  User, Admin - IT -...   
211  (Write - IT - Support Configurator Access, Adm...   
212  (Read - IT - Support - End  User, Write - IT -...   
213  (Read - IT - Support - End  User, Function - R...   
214  (Read - IT - Support - End  User, Admin - IT -...   
215  (Write - IT - Support Configurator Access, Fun...   
216  (Write - IT - Support Configurator Access, Adm...   
217  (Admin - IT - Middleware Administration Monito...   
218                  (Read - IT - Support - End  User)   
219         (Write - IT - Support Configurator Access)   
220                   (Function - RES_IB FRC R&C Risk)   
221   (Admin - IT - Middleware Administration Monitor)   
222  (Read - IT - Support - End  User, Write - IT -...   
223  (Read - IT - Support - End  User, Write - IT -...   
224  (Read - IT - Support - End  User, Write - IT -...   
225  (Write - IT - TriOptima Account Access, Write ...   
226  (Read - IT - Support - End  User, Write - IT -...   
227  (Read - IT - Support - End  User, Write - IT -...   
228  (Read - IT - Support - End  User, Function - R...   
229  (Write - IT - TriOptima Account Access, Write ...   
230  (Write - IT - TriOptima Account Access, Functi...   
231  (Write - IT - Support Configurator Access, Fun...   
232                  (Read - IT - Support - End  User)   
233            (Write - IT - TriOptima Account Access)   
234         (Write - IT - Support Configurator Access)   
235                   (Function - RES_IB FRC R&C Risk)   

                                           consequents  antecedent support  \
0     (Admin - IT - Middleware Administration Monitor)            0.022422   
1                     (Function - RES_IB FRC R&C Risk)            0.016442   
2     (Admin - IT - Middleware Administration Monitor)            0.030643   
3                    (Read - IT - Support - End  User)            0.016442   
4     (Admin - IT - Middleware Administration Monitor)  

In [54]:
## create rules_df to sort values by ascending confidence and lift confidence (i.e. ascending=1)
rules_df = rules.sort_values(['support', 'lift'], ascending=[True, True])

In [69]:
## sense check the data - no need to anonimise as no employee names are visible
rules_df.head(1)

antecedents  \
144  (Read - Offshore_Onshore - Trade Query and Agg...   
145  (Read - Offshore_Onshore - Trade Queries (Read...   
67       (Function - Validate Transactional Integrity)   
66       (Write - Middle Office - Controls Supervisor)   
68   (Write - Middle Office - Desktop Services Spec...   
69       (Function - Validate Transactional Integrity)   
143  (Read - Offshore_Onshore - Trade Queries (Read...   
146  (Read - Offshore_Onshore - Trade Query and Agg...   
194                 (Read - Finance - FRC (Read Only))   
191  (Function - Validate Transactional Integrity, ...   
195      (Function - Validate Transactional Integrity)   
190  (Read - Finance - FRC (Read Only), Write - Mid...   
142  (Read - Offshore_Onshore - Trade Queries (Read...   
147                            (Function - FCS Shared)   
192  (Read - Finance - FRC (Read Only), Function - ...   
193              (Write - Middle Office - Controls L1)   
96   (Write - Middle Office - Desktop Services Spec...   
97   (Write - Middle Office - Desktop Services Supe...   
80                   (Read - IT - Support - End  User)   
81             (Write - IT - TriOptima Account Access)   
180  (Write - IT - TriOptima Account Access, Functi...   
181                  (Read - IT - Support - End  User)   
204  (Write - IT - TriOptima Account Access, Write ...   
205                  (Read - IT - Support - End  User)   
225  (Write - IT - TriOptima Account Access, Write ...   
232                  (Read - IT - Support - End  User)   
86             (Write - IT - TriOptima Account Access)   
87          (Write - IT - Support Configurator Access)   
185  (Write - IT - TriOptima Account Access, Functi...   
188         (Write - IT - Support Configurator Access)   
..                                                 ...   
54          (Write - IT - Support Configurator Access)   
55                    (Function - RES_IB FRC R&C Risk)   
172  (Read - IT - Support - End  User, Write - IT -...   
173  (Read - IT - Support - End  User, Function - R...   
176         (Write - IT - Support Configurator Access)   
177                   (Function - RES_IB FRC R&C Risk)   
41                               (Write - Sales - FRC)   
40              (Function - Macro High Touch Americas)   
51                 (Function - Margin Management GCMM)   
50   (Read - Offshore_Onshore - Trade Queries (Read...   
44                  (Function - Macro High Touch EMEA)   
45                               (Write - Sales - FRC)   
82   (Read - Offshore_Onshore - Trade Queries (Read...   
83   (Read - Offshore_Onshore - Trade Query and Agg...   
48           (Function - Manage Business Facilitation)   
49   (Write - Offshore - Middle Office - Captive - ...   
37                          (Write - Sales - FRC APAC)   
36                  (Function - Macro High Touch APAC)   
78                   (Read - IT - Support - End  User)   
79          (Write - IT - Support Configurator Access)   
23   (Read - Offshore_Onshore - Trade Query and Agg...   
22                                 (Function - FCS IB)   
27                             (Function - FCS Shared)   
26   (Read - Offshore_Onshore - Trade Query and Agg...   
8    (Read - Offshore_Onshore - Trade Queries (Read...   
9               (Function - Asset Movements_Sec Setts)   
46   (Read - Offshore_Onshore - Trade Queries (Read...   
47           (Function - Manage Business Facilitation)   
103                              (Write - Sales - FRC)   
102                           (Write - Sales - CREDIT)   

                                           consequents  antecedent support  \
144  (Read - Offshore_Onshore - Trade Queries (Read...            0.032138   
145  (Read - Offshore_Onshore - Trade Query and Agg...            0.275785   
67       (Write - Middle Office - Controls Supervisor)            0.069507   
66       (Function - Validate Transactional Integrity)            0.032885   
68       (Function - Validate Transactional Integrity)  

In [68]:
## export the data into csv
rules_df.to_csv("example_function.csv")

## 5) Interpreting the Results

1.	We exported the results of the Apriori algorithm into an excel sheet.

2.	We filtered on items that start with {'Function as this will return all permission associations for user functions.

3.	We filtered on the function with the highest support out of this data set. This returned an item where the user function is 'Manage Business Facilitation' and the support for this is 0.041854 which means that approximately 4% of all users with access to the system are from the 'Manage Business Facilitation' function.

4.	Next we looked at the consequents for the 'Manage Business Facilitation' function – this is the permission 'Read - Offshore_Onshore - Trade Queries (Read Only)' and since the confidence is 0.7, this means that out of all users in the 'Manage Business Facilitation' function 70% of them will have this permission.

5.	Finally looking at the lift which is 2.538211 suggests that the function 'Manage Business Facilitation' and its consequent i.e. the permission 'Read - Offshore_Onshore - Trade Queries (Read Only)' are approximately 2.5 times more likely to occur together than if they were statistically independent. Hence this lift > 1 confirms that if the function of a user is 'Manage Business Facilitation', they are likely to have this permission to the system.

6.	We then filtered the Apriori results for items where there was a confidence greater than 50% between antecedents and consequents and hence we have come up with a list of permissions that can be recommended to new joiners in a particular function that require access to the system.

7.	There are of course limitations in our approach since our list contains only 21 functions whereas the total list of functions of users with access to the system is 168 (having looked at the original raw data). There are limitations to the Apriori algorithm in terms of computational processing time – there were instances where our laptops crashed completely and we were forced to increase the support. If we had run the algorithm with a lower support value, we would have been able to come up with recommendations for a wider range of user functions. 
